In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset


def char_2_one_hot(c: str, start_char='\t', end_char='\n', total_chars=28) -> list:
    '''
    char_2_one_hot()

    Converts a character to a one-hot encoding.

    Arguments:
     c: A character to be converted
     start_char: Special character to signify the beginning of a string
     end_char: Special character to signify the end of a string
    Returns:
     A list representing the one-hot encoding of the character 'c'
    '''
    if c == start_char:
        index = 0
    elif c == end_char:
        index = total_chars - 1
    elif c == ' ':  # padding char
        return [0] * total_chars
    else:
        # '+1' to accommodate the start character at index 0
        index = ord(c) - ord('a') + 1

    one_hot = [0] * total_chars
    one_hot[index] = 1
    return one_hot


def load_data(path='./Data/yob2018.txt', max_len=np.inf, start_char='\t', end_char='\n', total_chars=28) -> (np.ndarray, np.ndarray):
    '''
    load_data()

    This function loads the data from the filepath and processes it to
    remove names with special characters and convert all names to lowercase.

    Inputs are generated by right-shifting the names one position and adding the start_char to the beginning of the string.
    Outputs are generated by adding the end_chart to the end of the string.
    
    Inputs and outputs are then padded with ' ' so all names have uniform length.
    Then they are one-hot encoded.

    Arguments: 
     path - path to yob2018.txt data file
     start_char - special character to signify the beginning of a string
     end_char - special character to signify the end of a string
     total_chars - number of characters in the one-hot encoding (26 + 2 special characters)
    Returns: 
     inputs, outputs - processed arrays of inputs and outputs
    '''
    df = pd.read_csv('./Data/yob2018.txt', names=[
        'name', 'gender', 'occurrences'])

    # Filter out names with special characters and ensure names are not longer than max_len
    df = df[df['name'].str.match('^[a-zA-Z]+$') &
            (df['name'].str.len() <= max_len)]

    # convert the rest of the names to lowercase
    df['name'] = df['name'].str.lower()

    # max length
    N = df['name'].str.len().max()

    # Right shift inputs by 1 and add special character to the beginning
    df['inputs'] = start_char + df['name'].str[:-1]

    # Shift left by 1 to line up output next character with input last character and add special character to the end of the outputs
    # df['outputs'] = df['name'].str[1:] + end_char

    # Add special character to the end of the outputs
    df['outputs'] = df['name'] + end_char

    # pad names with spaces to make all names uniform length N plus the special character
    df['inputs'] = df['inputs'].apply(lambda name: name.ljust(N + 1, ' '))
    df['outputs'] = df['outputs'].apply(lambda name: name.ljust(N + 1, ' '))

    # One-hot encode
    inputs = np.array([[char_2_one_hot(c) for c in s] for s in df['inputs']])
    outputs = np.array([[char_2_one_hot(c) for c in s] for s in df['outputs']])

    return inputs, outputs


class LSTMModel(nn.Module):
    def __init__(self, max_len=np.inf, total_chars=28):
        super(LSTMModel, self).__init__()
        self.max_len = max_len
        self.lstm1 = nn.LSTM(input_size=total_chars,
                             hidden_size=128, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=128, hidden_size=128, batch_first=True)
        self.time_distributed = nn.Linear(128, total_chars)

    def forward(self, x, hidden=None):
        x, hidden1 = self.lstm1(x, hidden)
        x, hidden2 = self.lstm2(x, hidden1)
        x = self.time_distributed(x)
        return x, hidden2


def create_model(max_len):
    '''
    create_model()

    Arguments:
     max_len - Number of characters in the word (uniform length for all words)
    Returns:
      an LSTM network with the following architecture:

        LSTM (128 neurons)
        LSTM (128 neurons)
        TimeDistributed(Dense(total_chars neurons))
        SoftMax
    '''
    return LSTMModel(max_len=max_len)


def train_model(model, n_epochs):
    '''
    train_model()

    This function trains the model object for n_epochs

    Arguments:
     model - Model object from create_model()
     n_epochs - Number of epochs to train the model
    
    Returns:
     The trained model
    '''
    N = model.max_len
    # X, y = load_data(path='yob2018.txt', max_len=N) # use for submission
    X, y = load_data(max_len=N)
    X_tensor = torch.tensor(X, dtype=torch.float)
    y_tensor = torch.tensor(y, dtype=torch.long).argmax(
        dim=2)  # Convert one-hot to indices

    dataset = TensorDataset(X_tensor, y_tensor)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Training
    model.train()
    for epoch in range(n_epochs):
        total_loss = 0
        for batch_inputs, batch_targets in dataloader:
            optimizer.zero_grad()
            outputs, _ = model(batch_inputs)
            loss = criterion(outputs.transpose(1, 2), batch_targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        average_loss = total_loss / len(dataloader)
        print(f'Epoch {epoch + 1}/{n_epochs}, Average Loss: {average_loss:.4f}')

    return model


def save_model(model, filepath):
    '''
    save_model()

    Saves the model to the filepath
    '''
    torch.save(model.state_dict(), filepath)
    print(f"Model saved to {filepath}")


def load_trained_model(filepath):
    '''
    load_trained_model()

    Loads a trained model from the filepath
    '''
    model = create_model(None)
    model.load_state_dict(torch.load(filepath))
    model.eval()
    return model


def generate_name(M, N, start_char='\t', total_chars=28) -> str:
    '''
    generate_name()

    Generates a name using the trained LSTM model

    Arguments:
     M: The trained LSTM model
     N: Maximum length of names
     start_char: Special character to signify the beginning of a string
     total_chars: Number of characters in the one-hot encoding (26 + 2 special characters)
    Returns:
     A generated name
    '''
    M.eval()
    # Initial input with start character
    current_input = torch.tensor(
        [char_2_one_hot(start_char)], dtype=torch.float).unsqueeze(0)

    name = ''
    hidden = None  # initialize hidden state of LSTM

    for _ in range(N):
        # Forward pass through the model
        output, hidden = M(current_input, hidden)

        # Get the probability distribution from the last character output
        probabilities = torch.softmax(output[0, -1], dim=0).detach().numpy()

        # Sample a character from the probability distribution
        char_index = np.random.choice(total_chars, p=probabilities)

        # Convert index to character
        if char_index == 0:
            continue  # Resample if start character
        elif char_index == total_chars - 1:
            break  # Stop if end character
        else:
            char = chr(char_index + ord('a') - 1) if char_index > 0 else ' '
            name += char
            current_input = torch.tensor(
                [char_2_one_hot(char)], dtype=torch.float).unsqueeze(0)

    return name.strip()

In [9]:
file_id = '1Z1XErPV37IeumrRXuQRvRjmHfep1Kdxb'

In [1]:
import requests


def download_file_from_google_drive(file_id, destination):
    URL = "https://drive.google.com/uc?export=download"

    session = requests.Session()
    response = session.get(URL, params={'id': file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': file_id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None


def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)


file_id = '1Z1XErPV37IeumrRXuQRvRjmHfep1Kdxb'
destination = 'name_generator.pt'
download_file_from_google_drive(file_id, destination)

In [4]:
model = load_trained_model('name_generator.pt')

In [5]:
model

LSTMModel(
  (lstm1): LSTM(28, 128, batch_first=True)
  (lstm2): LSTM(128, 128, batch_first=True)
  (time_distributed): Linear(in_features=128, out_features=28, bias=True)
)